In [ ]:
import pandas as pd
import os
import ast
import numpy as np
import pdb

import google.generativeai as genai  # ✅ Only this import is needed for Gemini

genai.configure(api_key="")  # Replace with your actual API key

model = genai.GenerativeModel('gemini-2.0-flash')  # or 'gemini-1.5-flash'

instruction = "Think of you as a tourist guide in USA and you know many attractive places in california"
question = "What are the places to visit in california?"

full_prompt = instruction + "\n\n" +question

response = model.generate_content(full_prompt)

print(response.text)

Alright folks, welcome to California! Get ready for sunshine, stunning scenery, and a whole lot of fun. As your trusty tour guide, I'm going to give you a taste of the incredible diversity this state has to offer.  Prepare for some serious wanderlust!

**For the City Slicker & Culture Vulture:**

*   **San Francisco:**  No California trip is complete without it!

    *   **Golden Gate Bridge:** Walk, bike, or drive across for iconic views.  Especially magical at sunrise or sunset (if the fog cooperates!).
    *   **Alcatraz Island:** Book your tickets well in advance to tour this infamous former prison. The audio tour is excellent!
    *   **Fisherman's Wharf:** Sea lions, clam chowder in sourdough bowls, and souvenir shops galore.  A classic, if a bit touristy.
    *   **Chinatown:** One of the oldest and largest Chinatowns outside of Asia.  Incredible food, vibrant culture.
    *   **Golden Gate Park:**  Gardens, museums, the Japanese Tea Garden, bison... you could spend days here!
 